In [1]:
from datasets import *
from transformers import *
from tokenizers import *
import os
import json
import glob

2022-04-23 22:05:50.504685: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-23 22:05:50.504702: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
files=[]
for fi in glob.glob("Sankrit_Corpus/*.txt"):
    files.append(fi)

### Preparing the data-set

In [5]:
#files = ["file_train1.txt", "file_train2.txt","file_val.txt"]
#files =["temp/file.txt"]
dataset = load_dataset("text", data_files=files,split="train")
#dataset_tes = load_dataset("text", data_files=files_tes)

Resolving data files:   0%|          | 0/285 [00:00<?, ?it/s]

Using custom data configuration default-1ed83c5a8d86733d


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /home/msc2/.cache/huggingface/datasets/text/default-1ed83c5a8d86733d/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


In [6]:
d = dataset.train_test_split(test_size=0.15)
d["train"],d["test"]
#print(dataset['text'])

(Dataset({
     features: ['text'],
     num_rows: 2226951
 }),
 Dataset({
     features: ['text'],
     num_rows: 392992
 }))

In [7]:
for t in d["train"]["text"][:3]:
  print(t)
  print("="*50)

स्म्बद्धानि अक्षराणि[सम्पादयतु]
लालबाग-उद्यानवनं, विधानसौधः, उच्चन्यायालयः, कब्बनपार्क, नेहरुतारालयः विश्वेश्वरय्या वस्तुसंग्रहालयः टिप्पुसुल्तान्दुर्गम्, अलसूरुसरोवरं, तिप्पगोण्डहळ्ळिजलाशयः शङ्करमठः, रामकृष्णाश्रमः इत्यादयः।
भुक्तिमुक्तफलप्रदं निखिलाघसङ्घनिबर्हणं चन्द्रशेखरमाश्रये मम किं करिष्यति वै यमः ।


### Training the tokenizer

Next, we need to train our tokenizer. To do that, we need to write our dataset into text files, as that's what the tokenizers library require the input to be:

In [8]:
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer 
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset["text"]:
          print(t, file=f)

# save the training set to train.txt
dataset_to_text(d["train"], "train.txt")
# save the testing set to test.txt
dataset_to_text(d["test"], "test.txt")

## --

In [3]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
# training the tokenizer on the training set
files = ["train.txt"]
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 30_522
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = True

In [29]:
# initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()
# train the tokenizer
tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens)
# enable truncation up to the maximum 512 tokens
tokenizer.enable_truncation(max_length=max_length)

In [4]:
tokenizer

NameError: name 'tokenizer' is not defined

In [5]:
model_path = "pretrained-bert"

In [32]:
# make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)

In [33]:
# save the tokenizer  
tokenizer.save_model(model_path)

['pretrained-bert/vocab.txt']

In [7]:
# dumping some of the tokenizer config to config file, 
# including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

In [8]:
# when the tokenizer is trained and configured, load it as BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_path)

In [13]:
print(tokenizer("कर्मण्येवाधिकारस्ते मा फलेषु कदाचन । मा कर्मफलहेतुर्भुर्मा ते संगोऽस्त्वकर्मणि ॥", truncation=True, padding="max_length", max_length=max_length, return_special_tokens_mask=True))


{'input_ids': [2, 16384, 13748, 12181, 1465, 1817, 769, 26667, 248, 1465, 16230, 13452, 5646, 747, 218, 7662, 12682, 17093, 1307, 1314, 249, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [16]:
print("""token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,""")

token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,


### Tokenizing the Dataset

In [36]:
def encode_with_truncation(examples):
  """Mapping function to tokenize the sentences passed with truncation"""
  return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
  """Mapping function to tokenize the sentences passed without truncation"""
  return tokenizer(examples["text"], return_special_tokens_mask=True)

# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

# tokenizing the train dataset
train_dataset = d["train"].map(encode, batched=True)
# tokenizing the testing dataset
test_dataset = d["test"].map(encode, batched=True)
if truncate_longer_samples:
  # remove other columns and set input_ids and attention_mask as 
  train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
  test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
  test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
  train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
train_dataset, test_dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

(Dataset({
     features: ['text', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
     num_rows: 425
 }),
 Dataset({
     features: ['text', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
     num_rows: 76
 }))

In [37]:
#train_dataset['input_ids'][10]

In [38]:
#train_dataset['attention_mask'][10]

In [39]:
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result
# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
# remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
# might be slower to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True, batch_size=2_000,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts, batched=True, batch_size=2_000,
                                  num_proc=4, desc=f"Grouping texts in chunks of {max_length}")

### Loading the model

In [40]:
# initialize the model with the config
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = BertForMaskedLM(config=model_config)

### Pretraining

In [41]:
# initialize the data collator, randomly masking 20% (default is 15%) of the tokens for the Masked Language
# Modeling (MLM) task
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

In [45]:
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=2,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=4, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=4,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=4,  # evaluation batch size
    logging_steps=5,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=5,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 5
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [46]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [47]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text, special_tokens_mask.
***** Running training *****
  Num examples = 425
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 52


Step,Training Loss,Validation Loss
5,9.765100,9.921456
10,9.808400,9.717861
15,9.657600,9.589318
20,9.664200,9.434731
25,9.537900,9.326357
30,10.841400,9.238132
35,9.269000,9.198263
40,9.288000,9.182027
45,9.194300,9.106124
50,9.094200,9.112475


The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text, special_tokens_mask.
***** Running Evaluation *****
  Num examples = 76
  Batch size = 4
Saving model checkpoint to pretrained-bert/checkpoint-5
Configuration saved in pretrained-bert/checkpoint-5/config.json
Model weights saved in pretrained-bert/checkpoint-5/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text, special_tokens_mask.
***** Running Evaluation *****
  Num examples = 76
  Batch size = 4
Saving model checkpoint to pretrained-bert/checkpoint-10
Configuration saved in pretrained-bert/checkpoint-10/config.json
Model weights saved in pretrained-bert/checkpoint-10/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text, special_tokens_mask.
*

TrainOutput(global_step=52, training_loss=9.595280720637394, metrics={'train_runtime': 1461.7729, 'train_samples_per_second': 0.581, 'train_steps_per_second': 0.036, 'total_flos': 221355247564800.0, 'train_loss': 9.595280720637394, 'epoch': 1.97})

In [48]:
# load the model checkpoint
model = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-50"))
# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_path)

loading configuration file pretrained-bert/checkpoint-50/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pretrained-bert/checkpoint-50/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at pretrained-bert/checkpoint-50.
If your task is similar to the task the model of the checkpoi

In [49]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [50]:
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [58]:
examples = [
  "राजा कुमारो नेता च दूतः श्रेष्ठी [MASK] द्विजः  गजाध्यक्षश्च पूर्वाद्याः क्षत्रियाद्याश्चतुर्दिशम्",# चरो
  "लोमाशिकायाः खलु कक्कशब्दः पूर्णः स्वभावप्रभवः स तस्याः  येऽन्ये स्वराः ते प्रकृतेः [MASK] सर्वे च दीप्ता इति संप्रदिष्टाः",#अपेताः 
]
for example in examples:
      for prediction in fill_mask(example):
        print(f"{prediction['sequence']}, confidence: {prediction['score']}")
      print("="*50)

राजा कमारो नता च दतः शरषठी : दविजः गजाधयकषशच परवादयाः कषतरियादयाशचतरदिशम, confidence: 0.019571151584386826
राजा कमारो नता च दतः शरषठी, दविजः गजाधयकषशच परवादयाः कषतरियादयाशचतरदिशम, confidence: 0.005455114413052797
राजा कमारो नता च दतः शरषठी - दविजः गजाधयकषशच परवादयाः कषतरियादयाशचतरदिशम, confidence: 0.004073024727404118
राजा कमारो नता च दतः शरषठी इति दविजः गजाधयकषशच परवादयाः कषतरियादयाशचतरदिशम, confidence: 0.0024348858278244734
राजा कमारो नता च दतः शरषठीः दविजः गजाधयकषशच परवादयाः कषतरियादयाशचतरदिशम, confidence: 0.0021114652045071125
लोमाशिकायाः खल कककशबदः परणः सवभावपरभवः स तसयाः यऽनय सवराः त परकतः : सरव च दीपता इति सपरदिषटाः, confidence: 0.020804300904273987
लोमाशिकायाः खल कककशबदः परणः सवभावपरभवः स तसयाः यऽनय सवराः त परकतः, सरव च दीपता इति सपरदिषटाः, confidence: 0.005214350298047066
लोमाशिकायाः खल कककशबदः परणः सवभावपरभवः स तसयाः यऽनय सवराः त परकतः - सरव च दीपता इति सपरदिषटाः, confidence: 0.004102106671780348
लोमाशिकायाः खल कककशबदः परणः सवभावपरभवः स तसयाः यऽनय सवराः त परकतः इति सरव च दीपत